# Data processing

In [9]:
import json
base_path = "./Bevo-Bud-The-GPT/data"
with open(f'{base_path}/UTAustin_submissions.json', 'r') as f:
    submissions = json.load(f)

with open(f'{base_path}/UTAustin_comments.json', 'r') as f:
    comments = json.load(f)

sub_ids_map = {}
for index, sub in enumerate(submissions):
    if sub['id'] not in sub_ids_map:
        if "?" in sub["title"]:
            sub_ids_map[sub['id']] = sub
    else:
        print("Duplicate submission id: ", sub['id'], f" at index {index}")

com_ids_map = {}
for com in comments:
    com_ids_map[com['link_id']] = com

In [10]:
subs_coms_map = {}
for sub_id in sub_ids_map.keys():
    subs_coms_map[sub_id] = {
        "submission": sub_ids_map[sub_id],
        "comments": []
    }
ignore_comments_count = 0
for com_id in com_ids_map.keys():
    sub_id = com_id.split("_")[1]
    if sub_id in subs_coms_map:
        comment = com_ids_map[com_id]
        if comment["score"] > 1 and comment["parent_id"] == com_id:
            # filter all comments to get only the best comments
            subs_coms_map[sub_id]["comments"].append(com_id)
    else:
        ignore_comments_count += 1
print("Ignored comments: ", ignore_comments_count)

Ignored comments:  29104


In [15]:
# structing data into the form:
# replace "I" with "a student" for each response, grab answers with a certain score

# restructuring data into conversational format: 
# {"messages": [{"role": "system", "content": "You are helpful"}, {"role": "user", "content": "What's the capital of France?"}, {"role": "assistant", "content": "..."}]}

def format_reddit_post(submission, comment):
    conversation = {
        "messages": [
            {"role": "system", "content": "You help answer questions for UT Austin students"},
            {"role": "user", "content": submission['title'].strip().replace('\n', '') + " " + submission['selftext'].strip().replace('\n', '')},
            {"role": "assistant", "content": f"{comment['body']}"}

        ]
    }

    return conversation
get_question = lambda x: x['title'].strip().replace('\n', '') + " " + x['selftext'].strip().replace('\n', '')
get_answer = lambda x: x['body']
data = []

for sub_id in subs_coms_map.keys():
    submission = subs_coms_map[sub_id]["submission"]
    com_ids = subs_coms_map[sub_id]["comments"]
    for com_id in com_ids:
        comment = com_ids_map[com_id]
        conversation = {"question" : get_question(submission), "answer": get_answer(comment)}
        # conversation = format_reddit_post(submission, comment)
        data.append(conversation)


## Saving data

In [16]:
from sklearn.model_selection import train_test_split
import csv
import json

seed = 1
train, test = train_test_split(data, test_size=0.2, random_state=seed)

with open(f"{base_path}/train-v1.csv", "w") as f:
    writeer = csv.DictWriter(f, fieldnames=["question", "answer"])
    writeer.writeheader()
    writeer.writerows(train)

with open(f"{base_path}/test-v1.csv", "w") as f:
    writeer = csv.DictWriter(f, fieldnames=["question", "answer"])
    writeer.writeheader()
    writeer.writerows(test)